## Import

In [1]:
import keras
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.layers import Input, Embedding, LSTM, Dense, BatchNormalization, concatenate, multiply, add, Activation, Flatten
from keras.models import Model
from keras.callbacks import TensorBoard
from datetime import datetime

Using TensorFlow backend.


## Paramètrage de l'utilisation GPU
Les notebooks sont limité à 50% d'utilisation 

In [2]:
c = []
for d in ['/gpu:1', '/gpu:2', '/gpu:3']:
  with tf.device(d):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
    c.append(tf.matmul(a, b))
with tf.device('/cpu:0'):
  sum = tf.add_n(c)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(sum))

[[ 66.  84.]
 [147. 192.]]


In [3]:
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.8
#sess = tf.Session(config=config)

## Hyper Paramètre

In [4]:
batch_size=128
epochs=2
lera=0.001
activation="relu"

## Preprocessing

In [5]:
train_input1 = np.load("../Data/train-input_1.npy")
train_input2 = np.load("../Data/train-input_2.npy")
train_output1 = np.load("../Data/train-output_1.npy")
train_output2 = np.load("../Data/train-output_2.npy")
validation_input_1 = np.load("../Data/validation-input_1.npy")
validation_input_2 = np.load("../Data/validation-input_2.npy")
validation_output_1 = np.load("../Data/validation-output_1.npy")
validation_output_2 = np.load("../Data/validation-output_2.npy")

In [6]:
print("train_input1" + str(train_input1))
print("train_input1 shape" + str(train_input1.shape))
print("train_input2" + str(train_input2))
print("train_input2 shape" + str(train_input2.shape))
print("train_output1" + str(train_output1))
print("train_output1 shape" + str(train_output1.shape))
print("train_output2" + str(train_output2))
print("train_output2 shape" + str(train_output2.shape))
print("validation_input_1" + str(validation_input_1))
print("validation_input_1 shape" + str(validation_input_1.shape))
print("validation_input_2" + str(validation_input_2))
print("validation_input_2 shape" + str(validation_input_2.shape))
print("validation_output_1" + str(validation_output_1))
print("validation_output_1" + str(validation_output_1.shape))
print("validation_output_2" + str(validation_output_2))
print("validation_output_2 shape" + str(validation_output_2.shape))

train_input1[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
train_input1 shape(24000000, 884)
train_input2[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
train_input2 shape(24000000, 260)
train_output1[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
train_output1 shape(24000000, 260)
train_output2[[1.]
 [0.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
train_output2 shape(24000000, 1)
validation_input_1[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 1.]]
validation_input_1 shape(6000000, 884)
validation_input_2[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 0.

In [7]:
#input_1_shape = train_input1.shape
#input_2_shape = train_input2.shape
output_1_shape = train_output1.shape
output_2_shape = train_output2.shape

input_1_shape = 884,
input_2_shape = 260,

input_1 = Input(shape=(884,), name="intput_1")
input_2 = Input(shape=(260,), name="intput_2")
output_1 = Dense(260, activation='softmax', name="output_1")(input_1 )
output_2 = Dense(1, activation='sigmoid', name="output_2")(input_2)

W0714 13:18:23.020337 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 13:18:23.024506 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [8]:
# This embedding layer will encode the input sequence
C = Embedding(10000,  32 ,  input_length = input_1_shape)(input_1)
H = Embedding(10000,  16 ,  input_length = input_1_shape)(input_1)
#lstm_out = LSTM(32, return_sequences=True, input_shape=(-1, 24000000, 884, 32))(x)

W0714 13:18:23.054775 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [9]:
# Layer 1

Hx = concatenate([H, H])
F = Activation('sigmoid')(Hx)

I = Activation('sigmoid')(Hx)
L = Activation('tanh')(Hx)
IL = multiply([I, L])

C = multiply([C, F])
C = add([C, IL])

C = Activation('tanh')(C)
O = Activation('sigmoid')(Hx)
x = multiply([C, O])


In [10]:
y = Flatten()(x)
auxiliary_output = Dense(1, activation='sigmoid', name='output_2')(y)

test = Embedding(10000,  32 ,  input_length = (260))(input_2)

x = concatenate([x, test], axis=1)

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
#main_output = Dense(1, activation='sigmoid', name='output_1')(x)
x = Flatten()(x)
main_output = Dense(260, activation='softmax', name="output_1")(x)

In [11]:
model = Model(inputs=[input_1, input_2], outputs=[main_output, auxiliary_output])

In [ ]:
model.compile(optimizer='rmsprop',
              loss={'output_1': 'binary_crossentropy', 'output_2': 'binary_crossentropy'},
              loss_weights={'output_1': 1.0, 'output_2': 0.001})

# And trained it via:
#model.fit_generator({'main_input': train_input1, 'aux_input': train_input2},
#                      {'main_output': train_output1, 'aux_output': train_output2},
#                      samples_per_epoch=10000, steps_per_epoch=(train_samples/ batch_size),)
                        
#X_out = np.concatenate([train_output1, train_output2])
#r=len(train_output1)+len(train_output2)
#X_out = np.memmap("../Data/train-output_1.npy", shape=(r), mode='r+')
#X_out[len(train_output1):] = train_output2

#r=len(train_input1)+len(train_input2)
#X_in = np.memmap("../Data/train-input_1.npy", shape=(r), mode='r+')
#X_in[len(train_input1):] = train_input2

save_dir = os.path.join(os.getcwd(), 'res_logs')
date = datetime.today()
year = date.strftime("%Y")
month = date.strftime("%m")
day = date.strftime("%d")
hour = date.strftime("%H")
minute = date.strftime("%M")
model_name = "{}{}{}{}{}_test" \
    .format(year, month, day, hour, minute)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
callbacks = TensorBoard(log_dir=filepath)

#model.fit_generator([train_input1, train_input2], steps_per_epoch=15, epochs=1, verbose=1, 
#              callbacks=None, validation_data=X_out, 
#              validation_steps=None, class_weight=None, max_queue_size=10, workers=3, use_multiprocessing=True, 
#              shuffle=True, initial_epoch=0)

model.fit([train_input1, train_input2],
          [train_output1, train_output2],
          epochs=10, batch_size=4096, callbacks=[callbacks],
         validation_data=([validation_input_1, validation_input_2], [validation_output_1, validation_output_2]))

W0714 13:18:23.213797 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0714 13:18:23.231926 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0714 13:18:23.236143 140558498883328 deprecation.py:323] From /home/aargancointepas/.local/lib/python3.5/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0714 13:18:26.658596 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/ba

Train on 24000000 samples, validate on 6000000 samples


W0714 13:18:28.371874 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0714 13:18:28.373144 140558498883328 deprecation_wrapper.py:119] From /home/aargancointepas/.local/lib/python3.5/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/10
 8777728/24000000 [=========>....................] - ETA: 32:08 - loss: 0.0137 - output_1_loss: 0.0130 - output_2_loss: 0.6865

In [ ]:
# import subprocess

# bashCommand = "gcloud compute instances stop instance-2 -q --zone europe-west4-a"
# process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
# output, error = process.communicate()